In [1]:
from nltk.tag.stanford import POSTagger
import string
english_postagger = POSTagger('./stanford-postagger-2015-04-20/models/english-bidirectional-distsim.tagger', './stanford-postagger-2015-04-20/stanford-postagger.jar')

His father, XXX, ....
he is father of XXX

In [2]:
def remove_punct(line):
    temp = ''
    for c in line:
        if c == '\'':
            temp = temp + ' ' + c
        elif c not in string.punctuation:
            temp = temp + c
        else:
            temp = temp + ' '
    return temp

In [3]:
f = open('./text', 'r')
f2 = open('./result', 'r')
name = []

for line in f2:
    name.append(line.split('      ')[0])
    
f2.close()

keyword = ['father']

keyphrase = []  #without punctuations
original = []    # with punctuations

for line in f:
    if keyword[0] in line.lower():
        original.append(line)
        keyphrase.append(remove_punct(line))
f.close()

Pattern: His (adj) brother Eddard(,)   |   his (adj) brother, (adj) Eddard,  ... 

In [5]:
'''
keyword = ['brother']
His brother Stannis, Lord Tywin Lannister, Eddard Stark
text = '"It has a name, does it?" Her father sighed. "Ah, Arya. You have a wildness in you, child. \'The wolf blood,\' my father used to call it. Lyanna had a touch of it, and my brother Brandon more than a touch. It brought them both to an early grave." Arya heard sadness in his voice; he did not often speak of his father, or of the brother and sister who had died before she was born. "Lyanna might have carried a sword, if my lord father had allowed it. You remind me of her sometimes. You even look like her."'
keyphrase = []
original = []
original.append(text)
keyphrase.append(remove_punct(text))
'''

order = 0
for c in keyphrase:
    n = 0
    position = 0
    tokens = english_postagger.tag(c.split())
    #print tokens
    for element in tokens[0]:
        if keyword[0] in element:
            name = ''
            condition = ''         # ab for both behind and forword, abc without puctuations
            position = n
            #print position
            for i in range(position-1,-1,-1):
                if 'NN' in tokens[0][i] or 'JJ' in tokens[0][i]:
                    pass
                elif 'PRP$' in tokens[0][i]:
                    condition = condition + 'a'
                    prp = tokens[0][i][0]
                    start = i
                    break
                else:
                    break
            relation = keyword[0]
            for i in range(position+1,len(tokens[0])):
                if 'NNP' not in tokens[0][i]:
                    if position+1 != i:
                        condition = condition + 'b'
                    for j in range(position+1,i):
                        name = name + ' ' + tokens[0][j][0]
                    name = name.replace(' ','',1)
                    end = i
                    break
                if i == len(tokens[0]) - 1 and 'NNP' in tokens[0][i]:
                    condition = condition + 'b'
                    for j in range(position+1,i+1):
                        name = name + ' ' + tokens[0][j][0]
                    name = name.replace(' ','',1)
                    end = len(tokens[0]) - 1

            print tokens
            partition = original[order].split(',')
            for i in range(len(partition)):
                if keyword[0] in partition[i]:
                    temp = remove_punct(partition[i])
                    temp = temp.split(' ')
                    for j in range(len(temp)):
                        if keyword[0] in temp[j]:
                            if j < len(temp) - 1:             # his brother Stannis, ...
                                name_found = 0
                                condition = condition + 'c'
                            if j == len(temp) - 1:               # his brother, Stannis, ....
                                verb_found = 0
                                s = remove_punct(partition[i+1])
                                words = s.split(' ')
                                for obj in words:
                                    for t in tokens[0]:
                                        if obj in t:
                                            if t[1] == 'VBP' or t[1] == 'VB' or t[1] == 'VBD':
                                                verb_found = 1
                                                break
                                if verb_found == 0:
                                    condition = condition + 'c' 
                    break
                                
            print condition
            if condition == 'abc':
                print prp + '   ' + relation + '   ' + name + '\n'
            else:
                print 'result doesn\'t match\n'

        n = n + 1
        


        
    order = order + 1

[[(u'Brandon', u'NNP'), (u'Stark', u'NNP'), (u'had', u'VBD'), (u'bid', u'VBN'), (u'her', u'PRP$'), (u'wait', u'NN'), (u'as', u'RB'), (u'well', u'RB'), (u'I', u'PRP'), (u'shall', u'MD'), (u'not', u'RB'), (u'be', u'VB'), (u'long', u'JJ'), (u'my', u'PRP$'), (u'lady', u'NN'), (u'he', u'PRP'), (u'had', u'VBD'), (u'vowed', u'VBN'), (u'We', u'PRP'), (u'will', u'MD'), (u'be', u'VB'), (u'wed', u'VBN'), (u'on', u'IN'), (u'my', u'PRP$'), (u'return', u'NN'), (u'Yet', u'CC'), (u'when', u'WRB'), (u'the', u'DT'), (u'day', u'NN'), (u'came', u'VBD'), (u'at', u'IN'), (u'last', u'JJ'), (u'it', u'PRP'), (u'was', u'VBD'), (u'his', u'PRP$'), (u'brother', u'NN'), (u'Eddard', u'NNP'), (u'who', u'WP'), (u'stood', u'VBD'), (u'beside', u'IN'), (u'her', u'PRP'), (u'in', u'IN'), (u'the', u'DT'), (u'sept', u'NN')]]
abc
his   brother   Eddard

[[(u'Their', u'PRP$'), (u'host', u'NN'), (u'was', u'VBD'), (u'greater', u'JJR'), (u'than', u'IN'), (u'it', u'PRP'), (u'had', u'VBD'), (u'been', u'VBN'), (u'when', u'WRB'), (u'

Lord Eddard had a brother named Brandon

Bran's bastard brother Jon Snow   bc

In the end it came down to four; the Hound and his monstrous brother Gregor, Jaime Lannister the Kingslayer, and Ser Loras Tyrell, the youth they called the Knight of Flowers.



I will split the procedure of tokenizing the text and finding key informations 

dialog to detect relationships as father-son(daughter):

In [37]:
for line in original:
    partition = line.split('"')
    for i in range(len(partition)):
        if i%2 == 1 and keyword[0] in partition[i].lower():
            temp = partition[i].lower().replace(' ','')
            if ','+keyword[0]+',' in temp or keyword[0]+',' == temp[0:len(keyword[0])+1]:
                print line

  "Arya started it," Sansa said quickly, anxious to have the first word. "She called me a liar and threw an orange at me and spoiled my dress, the ivory silk, the one Queen Cersei gave me when I was betrothed to Prince Joffrey. She hates that I'm going to marry the prince. She tries to spoil everything, Father, she can't stand for anything to be beautiful or nice or splendid."

  "Please, Father," Sansa managed at last. "Please don't."

  "I didn't do anything wrong," Sansa pleaded with him. "I don't want to go back." She loved King's Landing; the pagaentry of the court, the high lords and ladies in their velvets and silks and gemstones, the great city with all its people. The tournament had been the most magical time of her whole life, and there was so much she had not seen yet, harvest feasts and masked balls and mummer shows. She could not bear the thought of losing it all. "Send Arya away, she started it, Father, I swear it. I'll be good, you'll see, just let me stay and I promise 